In [10]:
import wfdb
import matplotlib.pyplot as plt
import os

# === CONFIG ===
DATASET_PATH = r"C:\Users\DELL\Desktop\mit-bih-arrhythmia-database-1.0.0"
OUTPUT_PATH = r"C:\Users\DELL\Desktop\ecg_images_test"

os.makedirs(OUTPUT_PATH, exist_ok=True)

# Registros deseados
records_to_extract = ["100", "101", "102", "103"]

for record_name in records_to_extract:

    print(f"Procesando {record_name}...")

    # Cargar registro
    record = wfdb.rdrecord(os.path.join(DATASET_PATH, record_name))

    signal = record.p_signal[:,0]   # Lead 0
    fs = record.fs                  # Frecuencia de muestreo

    # Intervalo de 0 a 2 segundos
    start = 0
    end = int(fs * 2)  # 2 segundos

    segment = signal[start:end]

    # Graficar
    plt.figure(figsize=(6,3))
    plt.plot(segment)
    plt.title(f"Registro {record_name} — Intervalo 0–2 s")
    plt.tight_layout()

    output_file = os.path.join(OUTPUT_PATH, f"{record_name}_0to2s.png")
    plt.savefig(output_file)
    plt.close()

    print("Guardada:", output_file)

print("\n✔ Listo: Se generaron imágenes de 100, 101, 102, 103 (0–2 s).")


Procesando 100...
Guardada: C:\Users\DELL\Desktop\ecg_images_test\100_0to2s.png
Procesando 101...
Guardada: C:\Users\DELL\Desktop\ecg_images_test\101_0to2s.png
Procesando 102...
Guardada: C:\Users\DELL\Desktop\ecg_images_test\102_0to2s.png
Procesando 103...
Guardada: C:\Users\DELL\Desktop\ecg_images_test\103_0to2s.png

✔ Listo: Se generaron imágenes de 100, 101, 102, 103 (0–2 s).


In [11]:
import wfdb
import os

# === CONFIG ===
DATASET_PATH = r"C:\Users\DELL\Desktop\mit-bih-arrhythmia-database-1.0.0"
OUTPUT_IMG  = r"C:\Users\DELL\Desktop\ecg_images_test"
OUTPUT_LBL  = r"C:\Users\DELL\Desktop\ecg_labels_test"
os.makedirs(OUTPUT_LBL, exist_ok=True)

records_to_extract = ["100", "101", "102", "103"]
class_id = 0  # QRS

for record_name in records_to_extract:
    # Leer anotaciones del MIT-BIH
    ann = wfdb.rdann(os.path.join(DATASET_PATH, record_name), "atr")
    record = wfdb.rdrecord(os.path.join(DATASET_PATH, record_name))
    fs = record.fs

    # Intervalo 0–2 s
    start_sample = 0
    end_sample = int(fs * 2)

    # Selecciona picos QRS dentro del intervalo
    peaks = [p for p in ann.sample if start_sample <= p < end_sample]

    # Crear archivo label YOLO
    label_file = os.path.join(OUTPUT_LBL, f"{record_name}_0to2s.txt")
    with open(label_file, "w") as f:
        for p in peaks:
            # Normalizamos posición x de 0 a 1
            cx = (p - start_sample) / (end_sample - start_sample)
            cy, w, h = 0.5, 0.02, 1.0  # QRS centrado, ancho pequeño, alto completo
            f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    print(f"Labels generados para {record_name}: picos {peaks}")

print("\n✔ Listo: Labels YOLO generados para las imágenes 0–2 s.")


Labels generados para 100: picos [np.int64(18), np.int64(77), np.int64(370), np.int64(662)]
Labels generados para 101: picos [np.int64(7), np.int64(83), np.int64(396), np.int64(711)]
Labels generados para 102: picos [np.int64(68), np.int64(136), np.int64(410), np.int64(697)]
Labels generados para 103: picos [np.int64(21), np.int64(265), np.int64(575)]

✔ Listo: Labels YOLO generados para las imágenes 0–2 s.


In [13]:
import os
import shutil
from glob import glob

# === CONFIG ===
IMAGES_PATH = r"C:\Users\DELL\Desktop\images"   # Carpeta donde están tus .png
LABELS_PATH = r"C:\Users\DELL\Desktop\labels"   # Carpeta donde están tus .txt

# Crear subcarpetas train y val
for folder in ['train','val']:
    os.makedirs(os.path.join(IMAGES_PATH, folder), exist_ok=True)
    os.makedirs(os.path.join(LABELS_PATH, folder), exist_ok=True)

# Obtener todas las imágenes
image_files = glob(os.path.join(IMAGES_PATH, "*.png"))

if not image_files:
    print("⚠ No se encontraron imágenes en la carpeta:", IMAGES_PATH)
else:
    print(f"Se encontraron {len(image_files)} imágenes.")

# Separar 80% train y 20% val
train_count = int(len(image_files) * 0.8)

for i, img_file in enumerate(image_files):
    base_name = os.path.basename(img_file)
    label_file = os.path.join(LABELS_PATH, base_name.replace(".png",".txt"))
    
    # Verificar si existe el label
    if not os.path.exists(label_file):
        print(f"⚠ Label no encontrado para {base_name}, se omite.")
        continue
    
    # Mover archivos
    if i < train_count:
        shutil.move(img_file, os.path.join(IMAGES_PATH, "train", base_name))
        shutil.move(label_file, os.path.join(LABELS_PATH, "train", os.path.basename(label_file)))
    else:
        shutil.move(img_file, os.path.join(IMAGES_PATH, "val", base_name))
        shutil.move(label_file, os.path.join(LABELS_PATH, "val", os.path.basename(label_file)))

print("\n✔ Dataset organizado:")
print(f"- Train: {len(os.listdir(os.path.join(IMAGES_PATH,'train')))} imágenes")
print(f"- Val: {len(os.listdir(os.path.join(IMAGES_PATH,'val')))} imágenes")


Se encontraron 4173 imágenes.
⚠ Label no encontrado para 101_0.png, se omite.
⚠ Label no encontrado para 101_132.png, se omite.
⚠ Label no encontrado para 101_134.png, se omite.
⚠ Label no encontrado para 101_139.png, se omite.
⚠ Label no encontrado para 101_211.png, se omite.
⚠ Label no encontrado para 101_213.png, se omite.
⚠ Label no encontrado para 101_366.png, se omite.
⚠ Label no encontrado para 101_367.png, se omite.
⚠ Label no encontrado para 101_369.png, se omite.
⚠ Label no encontrado para 101_370.png, se omite.
⚠ Label no encontrado para 101_908.png, se omite.
⚠ Label no encontrado para 102_0.png, se omite.
⚠ Label no encontrado para 102_1.png, se omite.
⚠ Label no encontrado para 102_10.png, se omite.
⚠ Label no encontrado para 102_11.png, se omite.
⚠ Label no encontrado para 102_12.png, se omite.
⚠ Label no encontrado para 102_13.png, se omite.
⚠ Label no encontrado para 102_14.png, se omite.
⚠ Label no encontrado para 102_15.png, se omite.
⚠ Label no encontrado para 102_1

In [14]:
# Archivo: create_yaml.py (puedes ponerlo en tu notebook también)
yaml_path = r"C:\Users\DELL\Desktop\ecg_dataset.yaml"

with open(yaml_path, "w") as f:
    f.write(f"train: C:/Users/DELL/Desktop/images/train\n")
    f.write(f"val: C:/Users/DELL/Desktop/images/val\n\n")
    f.write("nc: 1\n")
    f.write("names: ['QRS']\n")

print("✔ YAML creado en:", yaml_path)


✔ YAML creado en: C:\Users\DELL\Desktop\ecg_dataset.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
print("✅ YOLOv8 listo con PyTorch")

# =======================
# 2️⃣ Entrenamiento
# =======================
# Ajusta epochs y batch según tu PC
results = model.train(
    data=r"C:/Users/DELL/Desktop/ecg_dataset.yaml",  # tu YAML
    epochs=50,
    imgsz=640,
    batch=16,          # disminuye si tu RAM/GPU es limitada
    name="qrs_detector"  # carpeta donde se guardarán los resultados
)

print("✅ Entrenamiento iniciado. Resultados en 'runs/train/qrs_detector/'")

✅ YOLOv8 listo con PyTorch
Ultralytics 8.3.229  Python-3.11.13 torch-2.9.1+cpu CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/DELL/Desktop/ecg_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=qrs_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_ma